# 44688-Data Analytics Capstone Project

## FDIP (Fire Department Incident Prediction)

##### 03/13/23 - 04/28/23

##### Debra D. "DeeDee" Walker

##### Northwest Missouri State University, Maryville MO 64468, USA

In [1]:
#Import the pandas framework to import and work with the dataset
import pandas as pd

#Set pandas to show all rows and not truncate
pd.set_option("display.max_rows", None)

#Import datetime so we can work with dates and times
import datetime as dt

#Import numpy
import numpy as np

#Data is accessed through the csv file download due to the limitations of the API on the site for this large dataset. The API endpoint limits hits and requires data to be requested by page
# Data was pulled from https://data.cityofnewyork.us/Public-Safety/Incidents-Responded-to-by-Fire-Companies/tm6d-hbzd on March 16, 2023
#Use the function read_csv from pandas and create a dataframe assigned to variable df
df = pd.read_csv("Incidents_Responded_to_by_Fire_Companies.csv", sep=",", dtype={"IM_INCIDENT_KEY": str, "FIRE_BOX": str, "INCIDENT_TYPE_DESC": str, "INCIDENT_DATE_TIME": object, 
                                                                                 "ARRIVAL_DATE_TIME": object, "UNITS_ONSCENE": float, "LAST_UNIT_CLEARED_DATE_TIME": object, "HIGHEST_LEVEL_DESC": str,
                                                                                "TOTAL_INCIDENT_DURATION": float, "ACTION_TAKEN1_DESC": str, "ACTION_TAKEN2_DESC": str, "ACTION_TAKEN3_DESC": str,
                                                                                "PROPERTY_USE_DESC": str, "STREET_HIGHWAY": str, "ZIP_CODE": str, "BOROUGH_DESC": str, "FLOOR": str, 
                                                                                 "CO_DETECTOR_PRESENT_DESC": str, "FIRE_ORIGIN_BELOW_GRADE_FLAG": str, "STORY_FIRE_ORIGIN_COUNT": str,
                                                                                "FIRE_SPREAD_DESC": str, "DETECTOR_PRESENCE_DESC": str, "AES_PRESENCE_DESC": str, "STANDPIPE_SYS_PRESENT_FLAG": str})

In [2]:
#Remove columns/features that we aren't using to reduce the size of the file or project concentration
df.drop(['FIRE_BOX', 'ARRIVAL_DATE_TIME', 'LAST_UNIT_CLEARED_DATE_TIME', 'STREET_HIGHWAY', 'FLOOR', 'CO_DETECTOR_PRESENT_DESC', 'FIRE_ORIGIN_BELOW_GRADE_FLAG',
                'STORY_FIRE_ORIGIN_COUNT', 'FIRE_SPREAD_DESC', 'DETECTOR_PRESENCE_DESC', 'AES_PRESENCE_DESC', 'STANDPIPE_SYS_PRESENT_FLAG', 'ACTION_TAKEN2_DESC',
                'ACTION_TAKEN3_DESC'], axis=1, inplace=True)

In [3]:
#convert INCIDENT_DATE_TIME from object to datetime format then print min & max
df["INCIDENT_DATE_TIME"] = pd.to_datetime(df["INCIDENT_DATE_TIME"], infer_datetime_format = True)
print (df["INCIDENT_DATE_TIME"].min())
print (df["INCIDENT_DATE_TIME"].max())

2013-01-01 00:00:20
2021-12-31 23:59:24


In [4]:
#filter by year only selecting 2017 - 2022 then print min & max to verify
df = df[(df["INCIDENT_DATE_TIME"]).dt.year.isin([2017,2018,2019,2020,2021])]
print (df["INCIDENT_DATE_TIME"].min())
print (df["INCIDENT_DATE_TIME"].max())

2017-01-01 00:00:22
2021-12-31 23:59:24


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2348968 entries, 2556 to 4158391
Data columns (total 10 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   IM_INCIDENT_KEY          object        
 1   INCIDENT_TYPE_DESC       object        
 2   INCIDENT_DATE_TIME       datetime64[ns]
 3   UNITS_ONSCENE            float64       
 4   HIGHEST_LEVEL_DESC       object        
 5   TOTAL_INCIDENT_DURATION  float64       
 6   ACTION_TAKEN1_DESC       object        
 7   PROPERTY_USE_DESC        object        
 8   ZIP_CODE                 object        
 9   BOROUGH_DESC             object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 197.1+ MB


In [6]:
#the file is large enough that info() doesn't report the null values so we have to force it.
#This is for record 1 through 1,599,999
df.iloc[:1600000].info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600000 entries, 2556 to 3409423
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   IM_INCIDENT_KEY          1600000 non-null  object        
 1   INCIDENT_TYPE_DESC       1600000 non-null  object        
 2   INCIDENT_DATE_TIME       1600000 non-null  datetime64[ns]
 3   UNITS_ONSCENE            1551650 non-null  float64       
 4   HIGHEST_LEVEL_DESC       1599927 non-null  object        
 5   TOTAL_INCIDENT_DURATION  1599929 non-null  float64       
 6   ACTION_TAKEN1_DESC       1600000 non-null  object        
 7   PROPERTY_USE_DESC        1600000 non-null  object        
 8   ZIP_CODE                 1599990 non-null  object        
 9   BOROUGH_DESC             1600000 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 134.3+ MB


In [7]:
#the file is large enough that info() doesn't report the null values so we have to force it.
#This is for record 1,600,000 through 2,348,968
df.iloc[1600000:2348969].info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748968 entries, 3409424 to 4158391
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   IM_INCIDENT_KEY          748968 non-null  object        
 1   INCIDENT_TYPE_DESC       748968 non-null  object        
 2   INCIDENT_DATE_TIME       748968 non-null  datetime64[ns]
 3   UNITS_ONSCENE            726885 non-null  float64       
 4   HIGHEST_LEVEL_DESC       748935 non-null  object        
 5   TOTAL_INCIDENT_DURATION  748417 non-null  float64       
 6   ACTION_TAKEN1_DESC       748968 non-null  object        
 7   PROPERTY_USE_DESC        748968 non-null  object        
 8   ZIP_CODE                 748966 non-null  object        
 9   BOROUGH_DESC             748968 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 62.9+ MB


In [8]:
df.describe()

,UNITS_ONSCENE,TOTAL_INCIDENT_DURATION
count,2.278535e+06,2.348346e+06
mean,1.942324e+00,1.452199e+03
std,1.823836e+00,3.092382e+03
min,1.000000e+00,0.000000e+00
25%,1.000000e+00,7.500000e+02
50%,1.000000e+00,1.054000e+03
75%,2.000000e+00,1.577000e+03
max,1.360000e+02,1.716464e+06


In [9]:
df["INCIDENT_TYPE_DESC"].value_counts()

300 - Rescue, EMS incident, other                                          890491
735A - Unwarranted alarm/defective condition of alarm system               126658
651 - Smoke scare, odor of smoke                                           118909
710 - Malicious, mischievous false call, other                             113326
412 - Gas leak (natural gas or LPG)                                        102712
522 - Water or steam leak                                                   96001
353 - Removal of victim(s) from stalled elevator                            91448
113 - Cooking fire, confined to container                                   87900
555 - Defective elevator, no occupants                                      56309
736 - CO detector activation due to malfunction                             49455
322 - Motor vehicle accident with injuries                                  43992
353S - Stalled occupied elevator-removal via hoistway door                  40300
445 - Arcing, sh

In [10]:
#Based on the code assign a new category
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('1'), 'INCIDENT_CATEGORY'] = 'FIRE'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('2'), 'INCIDENT_CATEGORY'] = 'OVERPRESSURE RUPTURE, EXPLOSION, OVERHEAT-NO FIRE'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('3'), 'INCIDENT_CATEGORY'] = 'RESCUE & EMS'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('4'), 'INCIDENT_CATEGORY'] = 'HAZARDOUS CONDITION-NO FIRE'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('5'), 'INCIDENT_CATEGORY'] = 'SERVICE CALL'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('6'), 'INCIDENT_CATEGORY'] = 'CANCELED, GOOD INTENT'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('7'), 'INCIDENT_CATEGORY'] = 'FALSE ALARM FALSE CALL'
df.loc[df['INCIDENT_TYPE_DESC'].str.startswith('8'), 'INCIDENT_CATEGORY'] = 'SEVERE WEATHER & NATURAL DISASTER'

In [11]:
#review value counts of new feature
df.iloc[:1600000].info(verbose=True)
df.iloc[1600000:2348969].info(verbose=True)
df["INCIDENT_CATEGORY"].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600000 entries, 2556 to 3409423
Data columns (total 11 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   IM_INCIDENT_KEY          1600000 non-null  object        
 1   INCIDENT_TYPE_DESC       1600000 non-null  object        
 2   INCIDENT_DATE_TIME       1600000 non-null  datetime64[ns]
 3   UNITS_ONSCENE            1551650 non-null  float64       
 4   HIGHEST_LEVEL_DESC       1599927 non-null  object        
 5   TOTAL_INCIDENT_DURATION  1599929 non-null  float64       
 6   ACTION_TAKEN1_DESC       1600000 non-null  object        
 7   PROPERTY_USE_DESC        1600000 non-null  object        
 8   ZIP_CODE                 1599990 non-null  object        
 9   BOROUGH_DESC             1600000 non-null  object        
 10  INCIDENT_CATEGORY        1600000 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 14

RESCUE & EMS                                         1157712
FALSE ALARM FALSE CALL                                387121
SERVICE CALL                                          240335
HAZARDOUS CONDITION-NO FIRE                           192945
CANCELED, GOOD INTENT                                 187170
FIRE                                                  171586
OVERPRESSURE RUPTURE, EXPLOSION, OVERHEAT-NO FIRE      11747
SEVERE WEATHER & NATURAL DISASTER                        352
Name: INCIDENT_CATEGORY, dtype: int64

In [12]:
df["HIGHEST_LEVEL_DESC"].value_counts()

11 - First Alarm                                     1852788
1 - More than initial alarm, less than Signal 7-5     479970
75 - All Hands Working                                  7424
00 - Complaint/Still                                    5329
7 - Signal 7-5                                          1973
22 - Second Alarm                                        507
0 - Initial alarm                                        433
33 - Third Alarm                                         164
2 - 2nd alarm                                            134
44 - Fourth Alarm                                         48
3 - 3rd alarm                                             36
55 - Fifth Alarm                                          19
5 - 5th alarm                                             14
4 - 4th alarm                                             14
66 - Sixth Alarm                                           4
77 - Seventh Alarm                                         3
88 - Eighth Alarm       

In [13]:
#Remove row with no data in highest level and duration
df.dropna(subset=["HIGHEST_LEVEL_DESC","TOTAL_INCIDENT_DURATION"], inplace = True)

In [14]:
#Based on the code assign a new category
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('0'), 'LEVEL_CATEGORY'] = '1'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('1'), 'LEVEL_CATEGORY'] = '1'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('2'), 'LEVEL_CATEGORY'] = '2'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('3'), 'LEVEL_CATEGORY'] = '3'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('4'), 'LEVEL_CATEGORY'] = '4'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('5'), 'LEVEL_CATEGORY'] = '5'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('6'), 'LEVEL_CATEGORY'] = '6'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('7'), 'LEVEL_CATEGORY'] = '7'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('8'), 'LEVEL_CATEGORY'] = '7'
df.loc[df['HIGHEST_LEVEL_DESC'].str.startswith('11'), 'LEVEL_CATEGORY'] = '7'

In [15]:
#review value counts of new feature
df.iloc[:1600000].info(verbose=True)
df.iloc[1600000:2348969].info(verbose=True)
df["LEVEL_CATEGORY"].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600000 entries, 2556 to 3409500
Data columns (total 12 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   IM_INCIDENT_KEY          1600000 non-null  object        
 1   INCIDENT_TYPE_DESC       1600000 non-null  object        
 2   INCIDENT_DATE_TIME       1600000 non-null  datetime64[ns]
 3   UNITS_ONSCENE            1551719 non-null  float64       
 4   HIGHEST_LEVEL_DESC       1600000 non-null  object        
 5   TOTAL_INCIDENT_DURATION  1600000 non-null  float64       
 6   ACTION_TAKEN1_DESC       1600000 non-null  object        
 7   PROPERTY_USE_DESC        1600000 non-null  object        
 8   ZIP_CODE                 1599992 non-null  object        
 9   BOROUGH_DESC             1600000 non-null  object        
 10  INCIDENT_CATEGORY        1600000 non-null  object        
 11  LEVEL_CATEGORY           1600000 non-null  object        
dt

7    1861687
1     485715
2        641
3        198
4         61
5         33
6          4
Name: LEVEL_CATEGORY, dtype: int64

In [16]:
#Drop converted features 
df.drop(['HIGHEST_LEVEL_DESC'], axis=1, inplace=True)
df.drop(['INCIDENT_TYPE_DESC'], axis=1, inplace=True)

In [17]:
df["ACTION_TAKEN1_DESC"].value_counts()

00 - Action taken, other                                   1182075
86 - Investigate                                            295391
64 - Shut down system                                       168721
11 - Extinguishment by fire service personnel               139700
45 - Remove hazard                                          114673
44 - Hazardous materials leak control & containment          96058
70 - Assistance, other                                       55295
22 - Rescue, remove from harm                                43495
31 - Provide first aid & check for injuries                  41496
93 - Cancelled en route                                      33278
82 - Notify other agencies.                                  33054
48 - Remove hazardous materials                              18870
302 - Provide Patient Care                                   18405
42 - HazMat detection, monitoring, sampling, & analysis      18174
53 - Evacuate area                                           1

In [18]:
#Drop feature since the most information is coming from incident type and highest level description
df.drop(["ACTION_TAKEN1_DESC"], axis=1, inplace=True)

In [19]:
df["PROPERTY_USE_DESC"].value_counts()

UUU - Undetermined                                          1386148
429 - Multifamily dwelling                                   458733
419 - 1 or 2 family dwelling                                 107393
960 - Street, other                                           97211
962 - Residential street, road or residential driveway        94598
961 - Highway or divided highway                              43686
963 - Street or road in commercial area                       25513
400 - Residential, other                                      19910
500 - Mercantile, business, other                             19050
174 - Rapid transit station                                   11161
599 - Business office                                         10736
449 - Hotel/motel, commercial                                  5710
000 - Property Use, other                                      5681
900 - Outside or special property, other                       4674
331 - Hospital - medical or psychiatric         

In [20]:
#Drop feature due to number of undetermined values
df.drop(['PROPERTY_USE_DESC'], axis=1, inplace=True)

In [21]:
df["BOROUGH_DESC"].value_counts()

4 - Brooklyn         687892
1 - Manhattan        569985
2 - Bronx            489721
5 - Queens           477394
3 - Staten Island    123347
Name: BOROUGH_DESC, dtype: int64

In [22]:
df["ZIP_CODE"].value_counts()

99999    54473
10456    41812
11206    36880
11212    36833
10029    36041
11207    35176
10002    32592
10451    31472
11201    28725
10467    28612
10457    28604
10027    28547
11208    28419
10453    28255
10458    27610
11226    27027
11211    26254
11221    26109
11233    25943
10460    23933
10035    23457
10454    22788
11236    22708
10025    22634
10452    22409
10009    22118
11203    21941
10455    21856
11234    21666
11101    20743
10314    20180
11213    20062
10021    19910
11224    19854
10003    19728
10472    19323
10468    19249
11691    19222
10469    18977
10473    18883
11385    18625
10032    18554
10016    17932
10001    17824
10463    17720
11216    17598
10466    17527
11229    17440
10459    17265
11235    17248
11373    17085
10019    17049
10031    16844
10026    16650
11230    16457
11368    16384
10462    16035
11432    15940
11434    15731
11210    15526
11220    15505
11217    15467
10011    15357
11214    15205
10036    15194
11237    15153
10304    1

In [23]:
#Fill in nan for units on scene with the mean and round feature to integer
df['UNITS_ONSCENE'].replace([np.nan], df['UNITS_ONSCENE'].mean(), inplace = True)
df['UNITS_ONSCENE'] = df['UNITS_ONSCENE'].round(0).astype(int)
df.iloc[:1600000].info(verbose=True)
df.iloc[1600000:2348969].info(verbose=True)
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600000 entries, 2556 to 3409500
Data columns (total 8 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   IM_INCIDENT_KEY          1600000 non-null  object        
 1   INCIDENT_DATE_TIME       1600000 non-null  datetime64[ns]
 2   UNITS_ONSCENE            1600000 non-null  int32         
 3   TOTAL_INCIDENT_DURATION  1600000 non-null  float64       
 4   ZIP_CODE                 1599992 non-null  object        
 5   BOROUGH_DESC             1600000 non-null  object        
 6   INCIDENT_CATEGORY        1600000 non-null  object        
 7   LEVEL_CATEGORY           1600000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 103.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 748339 entries, 3409501 to 4158390
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype   

,UNITS_ONSCENE,TOTAL_INCIDENT_DURATION
count,2.348339e+06,2.348339e+06
mean,1.944077e+00,1.452193e+03
std,1.796375e+00,3.092377e+03
min,1.000000e+00,0.000000e+00
25%,1.000000e+00,7.500000e+02
50%,1.000000e+00,1.054000e+03
75%,2.000000e+00,1.577000e+03
max,1.360000e+02,1.716464e+06


In [24]:
#assign counter/variable for while loop
b = 1

In [25]:
#99999 is not a real zipcode; Cross match to borough and impute most common zip
#Using df2, filter the borough and zip and add count values
while b < 6:
    df2 = df.groupby(["BOROUGH_DESC", "ZIP_CODE"]).size().reset_index(name='counts')
    # get the most frequent zip for a borough
    df3 = df2.loc[(df2["BOROUGH_DESC"].str.contains(str(b)))].sort_values(by="counts", ascending=False).nlargest(1,"counts")
    #assign value to variable
    x = df3.iloc[0,1]
    #Assign highest count value per borough to 99999 records
    df.loc[(df["BOROUGH_DESC"].str.startswith(str(b))) & (df["ZIP_CODE"] == '99999'),"ZIP_CODE"] = x
    b += 1

In [26]:
#verify that the 99999 is now replaced
df["ZIP_CODE"].value_counts()

10029    61481
10456    51660
11206    44808
11212    36833
11207    35176
10002    32592
10451    31472
11201    28725
10467    28612
10457    28604
10027    28547
11208    28419
10453    28255
11101    27767
10458    27610
11226    27027
11211    26254
11221    26109
11233    25943
10314    24413
10460    23933
10035    23457
10454    22788
11236    22708
10025    22634
10452    22409
10009    22118
11203    21941
10455    21856
11234    21666
11213    20062
10021    19910
11224    19854
10003    19728
10472    19323
10468    19249
11691    19222
10469    18977
10473    18883
11385    18625
10032    18554
10016    17932
10001    17824
10463    17720
11216    17598
10466    17527
11229    17440
10459    17265
11235    17248
11373    17085
10019    17049
10031    16844
10026    16650
11230    16457
11368    16384
10462    16035
11432    15940
11434    15731
11210    15526
11220    15505
11217    15467
10011    15357
11214    15205
10036    15194
11237    15153
10304    15085
11205    1

In [27]:
#Remove rows with no data
df.dropna(subset=["ZIP_CODE"], inplace = True)

In [28]:
#Add day of the week and hour of the day to the dataset
df['Day_of_week'] = df['INCIDENT_DATE_TIME'].dt.day_name()
df['Hour_of_day'] = df['INCIDENT_DATE_TIME'].dt.hour

In [29]:
#Convert incident duration from seconds to hours
df['TOTAL_INCIDENT_DURATION'] = df['TOTAL_INCIDENT_DURATION']/3600
df['TOTAL_INCIDENT_DURATION'] = df['TOTAL_INCIDENT_DURATION'].round(2)

In [30]:
#Print correlation between units on scene and incident duration
print(df['UNITS_ONSCENE'].corr(df['TOTAL_INCIDENT_DURATION']))

0.17843264235359615


In [31]:
#Convert duration to category 
Durations = [(df['TOTAL_INCIDENT_DURATION'] <= .25),
             (df['TOTAL_INCIDENT_DURATION'] >.25) & (df['TOTAL_INCIDENT_DURATION'] <= .50),
             (df['TOTAL_INCIDENT_DURATION'] > .50) & (df['TOTAL_INCIDENT_DURATION'] <= .75),
             (df['TOTAL_INCIDENT_DURATION'] > .75) & (df['TOTAL_INCIDENT_DURATION'] <= 1.00),
             (df['TOTAL_INCIDENT_DURATION'] > 1.00) & (df['TOTAL_INCIDENT_DURATION'] <= 2.00),
             (df['TOTAL_INCIDENT_DURATION'] > 2.00) & (df['TOTAL_INCIDENT_DURATION'] <= 3.00),
             (df['TOTAL_INCIDENT_DURATION'] > 3.00)]

Duration_Categories = ['<=15min','15min-30min','30min-45min','45min-1hr','1-2hr','2-3hr','3hr>']

df['TOTAL_INCIDENT_DURATION'] = np.select(Durations, Duration_Categories)

df['TOTAL_INCIDENT_DURATION'].value_counts()

15min-30min    966347
<=15min        917668
30min-45min    227390
45min-1hr      115928
1-2hr          105597
2-3hr            9446
3hr>             5954
Name: TOTAL_INCIDENT_DURATION, dtype: int64

In [32]:
#Review range of units on scene
df["UNITS_ONSCENE"].value_counts()

1      1442401
2       382455
3       196860
6       104556
5       102932
4        98877
7         8248
8         1803
12        1415
13        1211
11        1154
9         1014
14         950
10         900
15         714
16         487
17         322
18         269
19         228
20         197
21         131
22         113
26         108
24          99
23          85
25          83
27          74
28          55
31          42
30          42
34          40
29          40
32          31
33          31
36          26
37          25
35          24
70          24
38          22
40          16
49          15
44          15
48          15
43          12
39          12
41          12
55          11
66          11
42           9
63           9
76           8
45           8
50           7
51           6
61           6
53           6
57           4
67           4
68           4
58           4
90           3
54           3
52           3
47           3
56           3
60           3
46        

In [33]:
#Convert units on scene to category 
df.loc[df['UNITS_ONSCENE'] >= 7, 'UNITS_ONSCENE'] = 7
df['UNITS_ONSCENE'].value_counts()

1    1442401
2     382455
3     196860
6     104556
5     102932
4      98877
7      20249
Name: UNITS_ONSCENE, dtype: int64

In [34]:
#convert to object
df['UNITS_ONSCENE'] = df['UNITS_ONSCENE'].astype(str)

In [35]:
#Rename 7 value
df.loc[df['UNITS_ONSCENE'].str.startswith('7'), 'UNITS_ONSCENE'] = '7 or more'
df['UNITS_ONSCENE'].value_counts()

1            1442401
2             382455
3             196860
6             104556
5             102932
4              98877
7 or more      20249
Name: UNITS_ONSCENE, dtype: int64

In [36]:
#Check data verification before export
df.iloc[:1600000].info(verbose=True)
df.iloc[1600000:2348969].info(verbose=True)
df.describe()
df.head(n=10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600000 entries, 2556 to 3409508
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   IM_INCIDENT_KEY          1600000 non-null  object        
 1   INCIDENT_DATE_TIME       1600000 non-null  datetime64[ns]
 2   UNITS_ONSCENE            1600000 non-null  object        
 3   TOTAL_INCIDENT_DURATION  1600000 non-null  object        
 4   ZIP_CODE                 1600000 non-null  object        
 5   BOROUGH_DESC             1600000 non-null  object        
 6   INCIDENT_CATEGORY        1600000 non-null  object        
 7   LEVEL_CATEGORY           1600000 non-null  object        
 8   Day_of_week              1600000 non-null  object        
 9   Hour_of_day              1600000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 134.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 748330 

,IM_INCIDENT_KEY,INCIDENT_DATE_TIME,UNITS_ONSCENE,TOTAL_INCIDENT_DURATION,ZIP_CODE,BOROUGH_DESC,INCIDENT_CATEGORY,LEVEL_CATEGORY,Day_of_week,Hour_of_day
2556,63583742,2018-07-21 12:18:19,1,30min-45min,10475,2 - Bronx,RESCUE & EMS,7,Saturday,12
3281,63583743,2018-07-21 12:18:26,1,<=15min,11230,4 - Brooklyn,RESCUE & EMS,7,Saturday,12
3335,63584267,2018-07-21 14:16:40,2,<=15min,11204,4 - Brooklyn,HAZARDOUS CONDITION-NO FIRE,7,Saturday,14
3390,63584484,2018-07-21 15:07:51,1,15min-30min,11235,4 - Brooklyn,SERVICE CALL,7,Saturday,15
3607,63584485,2018-07-21 15:08:36,1,15min-30min,11208,4 - Brooklyn,RESCUE & EMS,7,Saturday,15
3904,63584710,2018-07-21 16:03:49,1,15min-30min,11693,5 - Queens,SERVICE CALL,7,Saturday,16
4337,63584716,2018-07-21 16:06:01,1,1-2hr,10464,2 - Bronx,RESCUE & EMS,7,Saturday,16
5867,63584720,2018-07-21 16:05:50,1,<=15min,10451,2 - Bronx,SERVICE CALL,7,Saturday,16
6290,63584722,2018-07-21 16:06:40,1,15min-30min,10468,2 - Bronx,RESCUE & EMS,7,Saturday,16
6317,63584727,2018-07-21 16:08:38,1,15min-30min,11357,5 - Queens,RESCUE & EMS,7,Saturday,16


In [37]:
#Export clean datset for use and loading on Github
df.to_csv('fdip_clean.csv', index=False)